# Setup


In [ ]:
!pip install accelerate -U
!pip install crcmod wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=31406 sha256=c4de9b30d916a36f7a029cd3faab0b817c73450c28347f0f19955ccb2d8505f4
  Stored in directory: /root/.cache/pip/wheels/85/4c/07/72215c529bd59d67e3dac29711d7aba1b692f543c808ba9e86
Successfully built crcmod


In [ ]:
!gsutil cp  gs://hien7613storage2/kgt5_data.pt /content/


Copying gs://hien7613storage2/kgt5_data.pt...
- [1 files][ 87.0 MiB/ 87.0 MiB]                                                
Operation completed over 1 objects/87.0 MiB.                                     


# Dataset

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import torch
from transformers import T5Tokenizer
import numpy as np


In [ ]:
%%capture

tokenizer = T5Tokenizer.from_pretrained('t5-small', padding=True)

def _tokenize( x):
    return tokenizer(x, return_tensors="pt")['input_ids'][0][:-1]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Hop1Index

In [ ]:

class Hop1Index:
    def __init__(self, triples, num_entities, key_col=0, max_context_size=64):
        self.max_context_size = max_context_size
        self.shuffle = False
        self.key_col = key_col
        self.triples = triples[triples[:, key_col].argsort()]
        keys, values_offset = np.unique(
            self.triples[:, key_col], axis=0, return_index=True
        )
        values_offset = np.append(values_offset, len(self.triples))
        self.keys = keys
        self.values_offset = values_offset

        self.key_to_start = -1 * np.ones(num_entities, dtype=int)
        self.key_to_start[keys] = values_offset[:-1]
        self.key_to_end = -1 * np.ones(num_entities, dtype=int)
        self.key_to_end[keys] = values_offset[1:]

    def __getitem__(self, item, rel_id=None):
        start = self.key_to_start[item]
        end = self.key_to_end[item]
        context = self.triples[start:end, [1, 2 - self.key_col]]
        if rel_id is not None:
            context = context[context[:,0] == rel_id][:,1]
        if len(context) > self.max_context_size:
            ids = np.random.choice(len(context), self.max_context_size, replace=False)
            context = context[ids]
        if self.shuffle:
            np.random.shuffle(context)
        return context

    def get_context(self, item, rel_id=None):
        return self.__getitem__(item, rel_id)



## KGCDataset

In [ ]:
kgt5_data = torch.load('/content/kgt5_data.pt')

<ipython-input-7-d1e9fbb2f084>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  kgt5_data = torch.load('/content/kgt5_data.pt')


In [ ]:
from numpy import pi
import warnings

class RotatE:
    def __init__(self, k, max_rel_size=None, entity_embedding=None, relation_embedding=None):
        self.internal_k = 2 * k
        self.max_rel_size = max_rel_size
        self.entity_embedding = entity_embedding
        self.relation_embedding = relation_embedding

    def __call__(self, e_s_id, e_p_id):
        e_s = self.entity_embedding[e_s_id]
        e_p = self.relation_embedding[e_p_id]
        e_s_real, e_s_img = torch.chunk(e_s, 2, axis=0)
        theta_pred, _ = torch.chunk(e_p, 2, axis=0)

        embedding_range = (6 / (self.internal_k * self.max_rel_size)) ** 0.5
        e_p_real = torch.cos(theta_pred / (embedding_range / pi))
        e_p_img = torch.sin(theta_pred / (embedding_range / pi))

        e_o_real = e_s_real * e_p_real - e_s_img * e_p_img
        e_o_img = e_s_real * e_p_img + e_s_img * e_p_real
        return torch.cat([e_o_real, e_o_img], axis=0)

rotatE = RotatE(k=350, entity_embedding=kgt5_data['RotatE_ent_emb'], relation_embedding=kgt5_data['RotatE_rel_emb'], max_rel_size=237)


In [ ]:
import numpy as np
from torch.utils.data import Dataset
from typing import Dict, Optional, Union, Tuple, List
import random


class KGCDataset(Dataset):
    def __init__(self, num_ents=14541, structal_model=None):
        self.num_ents = num_ents
        self.structal_model = structal_model
        index = 3000
        # Fb15k wn18rr
        self.id_triplets ={
            'train': kgt5_data['train_triplet_id'],
            'valid': kgt5_data['valid_triplet_id'],
            'test': kgt5_data['test_triplet_id']
        }
        self.tokens_triplets ={
            'train': kgt5_data['train_triplet_tokens'],
            'valid': kgt5_data['valid_triplet_tokens'],
            'test': kgt5_data['test_triplet_tokens']
        }
        self.decs_triplets ={
            'train': kgt5_data['train_triplet_decs'],
            'valid': kgt5_data['valid_triplet_decs'],
            'test': kgt5_data['test_triplet_decs']
        }

        self.get_neigs_0 ={
            'train': Hop1Index(self.id_triplets['train'], self.num_ents, 0),
            'valid': Hop1Index(
                self.id_triplets['valid'],
                self.num_ents, 0),
            'test': Hop1Index(
                kgt5_data['test_triplet_id'],
                self.num_ents, 0)
        }
        self.get_neigs_2 ={
            'train': Hop1Index(self.id_triplets['train'], self.num_ents, 2),
            'valid': Hop1Index(
                self.id_triplets['valid'],
                self.num_ents, 2),
            'test': Hop1Index(
                kgt5_data['test_triplet_id'],
                self.num_ents, 2)
        }

        self.mask_token = _tokenize('<extra_id_90>')
        self.eos_token = torch.tensor([tokenizer.eos_token_id])
        self.zero_neig_embedding = torch.zeros([512])

        self.predict_head_token = _tokenize('predict head :')
        self.predict_tail_token = _tokenize('predict tail :')
        self.start_decs_token = _tokenize('[')
        self.end_decs_token = _tokenize(']')
        self.inversion_token = _tokenize('inversion of ')
        self.empty_token = torch.tensor([], dtype=torch.int)
        self.set_ent_id = set(range(self.num_ents))
        self.p_dropout = 0. # 0.2 when training

    def __getitem__(self, idx):
        return self.get(idx, split=self.split)
    def __len__(self, split='train'):
        return len(self.tokens_triplets[split])

    def get(self, idx: int, split: str = "train", full_mask_part_idx=None):
        head_lbl, relation, tail_lbl = self.tokens_triplets[split][idx]
        head_id, rel_id, tail_id = self.id_triplets[split][idx]
        head_decs, tail_decs = self.decs_triplets[split][idx]

        if full_mask_part_idx is None:
          full_mask_part_idx = 2 if random.randint(0, 1) else 0

        inversion = False

        if full_mask_part_idx:
          source = [
              self.predict_tail_token if not inversion else self.predict_head_token,
              head_lbl,
              self.start_decs_token,
              head_decs,
              self.end_decs_token,
              self.inversion_token if inversion else self.empty_token,
              relation,
          ]
          target = [tail_lbl]
          label_id = tail_id
          # filter_id = torch.cat([set_neig.get_context(head_id, rel_id) for set_neig in self.get_neigs_0.values()])
          neighboors_0 = self.get_neigs_0[split][head_id]
          neighboors_0 = neighboors_0[(neighboors_0[:,0]!=rel_id) | (neighboors_0[:,1]!=tail_id)]
          neighboors_2 = self.get_neigs_2[split][head_id]
          neighboors_2 = neighboors_2[(neighboors_2[:,0]!=rel_id) | (neighboors_2[:,1]!=tail_id)]
        else:
          source = [
              self.predict_head_token if not inversion else self.predict_tail_token,
              tail_lbl,
              self.start_decs_token,
              tail_decs,
              self.end_decs_token,
              self.inversion_token if inversion else self.empty_token,
              relation,
          ]
          target = [head_lbl]
          label_id = head_id
          # filter_id = torch.cat([set_neig.get_context(tail_id, rel_id) for set_neig in self.get_neigs_2.values()])
          neighboors_0 = self.get_neigs_0[split][tail_id]
          neighboors_0 = neighboors_0[(neighboors_0[:,0]!=rel_id) | (neighboors_0[:,1]!=head_id)]
          neighboors_2 = self.get_neigs_2[split][tail_id]
          neighboors_2 = neighboors_2[(neighboors_2[:,0]!=rel_id) | (neighboors_2[:,1]!=head_id)]

        target_ent_embeddings = []
        neighboors_embeddings = []
        for rel_n_id, ent_n_id in neighboors_0:
          if ent_n_id >= 14505:
            continue
          ent_n_embedding = self.structal_model.entity_embedding[ent_n_id]
          rel_n_embedding = self.structal_model.relation_embedding[rel_n_id]
          target_ent_embedding = self.structal_model(ent_n_id, rel_n_id)
          neighboors_embeddings.append(torch.cat([ent_n_embedding, rel_n_embedding]))
          target_ent_embeddings.append(target_ent_embedding)
        for rel_n_id, ent_n_id in neighboors_2:
          if ent_n_id >= 14505:
            continue
          ent_n_embedding = self.structal_model.entity_embedding[ent_n_id]
          rel_n_embedding = self.structal_model.relation_embedding[rel_n_id]
          target_ent_embedding = self.structal_model(ent_n_id, rel_n_id)
          neighboors_embeddings.append(torch.cat([ent_n_embedding, -rel_n_embedding]))
          target_ent_embeddings.append(target_ent_embedding)

        if len(neighboors_embeddings):
          neighboors_embeddings = torch.stack(neighboors_embeddings)
          target_ent_embeddings = torch.stack(target_ent_embeddings)
          neighboors_embeddings_mask = torch.ones(len(neighboors_embeddings))
        else:
          neighboors_embeddings_mask = torch.zeros([1])
          neighboors_embeddings = torch.zeros([1, 700*2])
          target_ent_embeddings = torch.zeros([1, 700])


        source.append(self.eos_token)
        target.append(self.eos_token)
        source = torch.cat(source)
        target = torch.cat(target)

        attention_mask = torch.ones_like(source)
        rand = torch.rand_like(attention_mask.float())
        dropout = torch.logical_not(rand < self.p_dropout).long()
        dropout[(source == self.start_decs_token[0]) | (source == self.end_decs_token[0])] = 1
        dropout[:4]=1
        inversion_len = len(self.inversion_token if inversion else self.empty_token)
        relation_len = len(relation)
        dropout[-relation_len-inversion_len:-relation_len]=1
        attention_mask = attention_mask * dropout


        output = {
            "input_ids": source,
            "attention_mask": attention_mask,
            "labels": target,
            # 'neighboors_embeddings': neighboors_embeddings,
            # 'neighboors_embeddings_mask': neighboors_embeddings_mask,
            # 'target_ent_embeddings': target_ent_embeddings,
            # 'triplet': self.id_triplets[split][idx],
        }
        return output

dataset = KGCDataset(num_ents=14541, structal_model=rotatE)

ext_get_neigs_0 ={
    'train': Hop1Index(
        kgt5_data['train_triplet_id'],
        dataset.num_ents, 0, max_context_size=1e10),
    'valid': Hop1Index(
        kgt5_data['valid_triplet_id'],
        dataset.num_ents, 0, max_context_size=1e10),
    'test': Hop1Index(
        kgt5_data['test_triplet_id'],
        dataset.num_ents, 0, max_context_size=1e10),
}

ext_get_neigs_2 ={
    'train': Hop1Index(
        kgt5_data['train_triplet_id'],
        dataset.num_ents, 2, max_context_size=1e10),
    'valid': Hop1Index(
        kgt5_data['valid_triplet_id'],
        dataset.num_ents, 2, max_context_size=1e10),
    'test': Hop1Index(
        kgt5_data['test_triplet_id'],
        dataset.num_ents, 2, max_context_size=1e10),
}

# get all ground truth
def get_neigs2(ent_id, rel_id):
  n_train = ext_get_neigs_2['train'].__getitem__(ent_id, rel_id)
  n_valid = ext_get_neigs_2['valid'].__getitem__(ent_id, rel_id)
  n_test = ext_get_neigs_2['test'].__getitem__(ent_id, rel_id)
  return [n_train, n_valid, n_test]
# get all ground truth
def get_neigs0(ent_id, rel_id):
  n_train = ext_get_neigs_0['train'].__getitem__(ent_id, rel_id)
  n_valid = ext_get_neigs_0['valid'].__getitem__(ent_id, rel_id)
  n_test = ext_get_neigs_0['test'].__getitem__(ent_id, rel_id)
  return [ n_train, n_valid, n_test]

class SplitDatasetWrapper:
    def __init__(self, dataset, split, full_mask_part_idx=None):
        self.dataset = dataset
        self.split = split
        self.full_mask_part_idx = full_mask_part_idx
    def __getitem__(self, idx):
        return self.dataset.get(idx, self.split, self.full_mask_part_idx)
    def __len__(self):
        return self.dataset.__len__(split=self.split)

train_dataset = SplitDatasetWrapper(dataset, split="train")
valid_dataset = SplitDatasetWrapper(dataset, split="valid")
test_dataset = SplitDatasetWrapper(dataset, split="test")

head_test_dataset = SplitDatasetWrapper(dataset, split="test", full_mask_part_idx=0)
tail_test_dataset = SplitDatasetWrapper(dataset, split="test", full_mask_part_idx=2)

head_valid_dataset = SplitDatasetWrapper(dataset, split="valid", full_mask_part_idx=0)
tail_valid_dataset = SplitDatasetWrapper(dataset, split="valid", full_mask_part_idx=2)

head_train_dataset = SplitDatasetWrapper(dataset, split="train", full_mask_part_idx=0)
tail_train_dataset = SplitDatasetWrapper(dataset, split="train", full_mask_part_idx=2)

# model

In [ ]:
from transformers import T5ForConditionalGeneration

model_name='t5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name)
# model.to('cuda')

print('')


In [ ]:
# # !rm /content/kgt5_rotatE_x5.pt
# !gsutil cp  gs://hien7613storage2/kgt5_rotatE_x11.pt /content/
# state_dict = torch.load('kgt5_rotatE_x11.pt', map_location=torch.device('cpu'))
# model.load_state_dict(state_dict)


## DataCollatorForSeq2Seq

In [ ]:
from torch.nn.utils.rnn import pad_sequence

class DataCollatorForSeq2Seq:
    model= None
    padding= True
    max_length= None
    pad_to_multiple_of=None
    label_pad_token_id= -100
    data_names = None
    def __init__(self, tokenizer, model=None, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100,data_names=None):
        self.tokenizer = tokenizer
        self.model = model
        self.data_names = data_names
        self.label_pad_token_id = label_pad_token_id

    def __call__(self, features):
        features2 = {}
        for name in self.data_names:
          if name == 'triplet':
            continue
          if name in ['labels','filter_id']:
            padding_value=self.label_pad_token_id
          else:
            padding_value=self.tokenizer.pad_token_id
          x_features = [feature[name] for feature in features]
          features2[name] = torch.nn.utils.rnn.pad_sequence(x_features, batch_first=True, padding_value=padding_value)
        if self.model is not None and hasattr(self.model, "prepare_decoder_input_ids_from_labels"):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features2["labels"])
            features2["decoder_input_ids"] = decoder_input_ids
        return features2


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, data_names=list(train_dataset[0].keys()))


In [ ]:
train_dataset[0]

{'input_ids': tensor([ 9689,   819,     3,    10,  5750,   784,    71, 20237,    19,     3,
             9,   607,    13,   789,    16,    84,   579,    19,  1213,    57,
             8,   151,    11,  8675,    79, 11924,     6,    11, 12748,    13,
           538,    33,     3,     9,     3,     2,   121, 15727,  1052,     2,
          1686,  1066,   145, 19852,  6563,    26,     5,    86,   941,   648,
             8,  4903,    13,     3,     9, 20237,    19,    92,  5871,  1643,
            12,     3,     9,   789,    84, 17981,     7,     3,     9, 28397,
             3,   908,  1128,   684,   607,    13,   789,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]),
 'labels': tensor([19169,   152,  5750,     1])}

In [ ]:
data = data_collator([train_dataset[0]])

#train

In [ ]:
from transformers import Seq2SeqTrainingArguments, TrainingArguments
from transformers import Seq2SeqTrainer
batch_size= 32*4

args = Seq2SeqTrainingArguments(
    "kgt5-rotatE",
    dataloader_num_workers=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    num_train_epochs=100,
    do_eval=True,

    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy='epoch',

    learning_rate=1e-4,
    # torch_compile=True,
    fp16=True,

    tf32=True,
    report_to='none',
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer
from transformers import EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
)

In [ ]:
!gsutil cp /content/kgt5-rotatE/checkpoint-25512.zip gs://hien7613storage2/

Copying file:///content/kgt5-rotatE/checkpoint-25512.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][618.6 MiB/618.6 MiB]                                                
Operation completed over 1 objects/618.6 MiB.                                    


In [ ]:
trainer.train(resume_from_checkpoint='/content/kgt5-rotatE/checkpoint-12756')
# baaed1dc0ef02b02dff291c8e0cfacf571bff2f9

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pleas

Epoch,Training Loss,Validation Loss
7,1.785700,1.514204
8,1.719100,1.452341
9,1.655400,1.419580
10,1.609600,1.393566
11,1.570200,1.364775
12,1.538500,1.371552
13,3.801800,4.198689
14,5.522400,4.242440


KeyboardInterrupt: 

In [ ]:
trainer.train()
# baaed1dc0ef02b02dff291c8e0cfacf571bff2f9

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.996500,2.332599
2,2.492400,2.038121
3,2.262600,1.857196
4,2.093300,1.739745
5,1.968800,1.647498
6,1.867400,1.564553
7,1.785700,1.514724
8,1.719600,1.453535
9,1.656200,1.420135
10,1.609800,1.395179


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()
# baaed1dc0ef02b02dff291c8e0cfacf571bff2f9

Epoch,Training Loss,Validation Loss
6,1.867400,1.574712


{'eval_loss': 1.5747116804122925}

In [ ]:
import torch

trainer.model.eval()
model_state_dict = trainer.model.state_dict()
torch.save(model_state_dict, '/content/kgt5_rotatE_x12.pt')

!gsutil -o GSUtil:parallel_composite_upload_threshold=150M cp  /content/kgt5_rotatE_x12.pt gs://hien7613storage2/

# New Eval

## setup eval

In [ ]:
# ent2text
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/entity2text.txt'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entity2text.txt
path = "/content/entity2text.txt"

ent2text = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    ent, text = line.strip().split('\t')
    ent2text[ent] = _tokenize(text)

# ent2id
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/entities.txt'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entities.txt
path = "/content/entities.txt"

ent2id = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    # print(line.strip().split('\t'))
    ent = line.strip().split('\t')[0]
    ent2id[ent] = int(i)

entid2text = [0]*len(ent2id)
for ent in tqdm(ent2id):
  entid2text[ent2id[ent]] = [0] + ent2text[ent].tolist() + [1]

ent_name_decode_list = []
for target in tqdm(entid2text):
  ent_name_decode_list.append(tokenizer.decode(target[1:-1]))

--2024-04-25 11:35:46--  https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entity2text.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zjunlp/Relphormer/main/dataset/fb15k-237/entity2text.txt [following]
--2024-04-25 11:35:46--  https://raw.githubusercontent.com/zjunlp/Relphormer/main/dataset/fb15k-237/entity2text.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 392383 (383K) [text/plain]
Saving to: ‘entity2text.txt’

entity2text.txt     100%[===================>] 383.19K  --.-KB/s    in 0.005s  

2024-04-25 11:35:47 (78.3 MB/s) - ‘entity2text.txt’ saved [392383/392383]


Processing lines:   0%|          | 0/14951 [00:00<?, ?it/s]

--2024-04-25 11:35:49--  https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entities.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zjunlp/Relphormer/main/dataset/fb15k-237/entities.txt [following]
--2024-04-25 11:35:49--  https://raw.githubusercontent.com/zjunlp/Relphormer/main/dataset/fb15k-237/entities.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142414 (139K) [text/plain]
Saving to: ‘entities.txt’

entities.txt        100%[===================>] 139.08K  --.-KB/s    in 0.002s  

2024-04-25 11:35:49 (60.7 MB/s) - ‘entities.txt’ saved [142414/142414]



Processing lines:   0%|          | 0/14541 [00:00<?, ?it/s]

  0%|          | 0/14541 [00:00<?, ?it/s]

  0%|          | 0/14541 [00:00<?, ?it/s]

In [ ]:
from typing import Dict, List
class Trie(object):
    def __init__(self, sequences: List[List[int]] = []):
        self.trie_dict = {}
        self.len = 0
        if sequences:
            for sequence in sequences:
                Trie._add_to_trie(sequence, self.trie_dict)
                self.len += 1
        self.append_trie = None
        self.bos_token_id = None
    def append(self, trie, bos_token_id):
        self.append_trie = trie
        self.bos_token_id = bos_token_id
    def add(self, sequence: List[int]):
        Trie._add_to_trie(sequence, self.trie_dict)
        self.len += 1
    def get(self, prefix_sequence: List[int]):
        return Trie._get_from_trie(prefix_sequence, self.trie_dict, self.append_trie, self.bos_token_id)
    @staticmethod
    def load_from_dict(trie_dict):
        trie = Trie()
        trie.trie_dict = trie_dict
        trie.len = sum(1 for _ in trie)
        return trie
    @staticmethod
    def _add_to_trie(sequence: List[int], trie_dict: Dict):
        if sequence:
            if sequence[0] not in trie_dict:
                trie_dict[sequence[0]] = {}
            Trie._add_to_trie(sequence[1:], trie_dict[sequence[0]])
    @staticmethod
    def _get_from_trie(
        prefix_sequence: List[int],
        trie_dict: Dict,
        append_trie=None,
        bos_token_id: int = None,
    ):
        if len(prefix_sequence) == 0:
            output = list(trie_dict.keys())
            if append_trie and bos_token_id in output:
                output.remove(bos_token_id)
                output += list(append_trie.trie_dict.keys())
            if len(output) == 0:
                return [0]
            return output
        elif prefix_sequence[0] in trie_dict:
            return Trie._get_from_trie(
                prefix_sequence[1:],
                trie_dict[prefix_sequence[0]],
                append_trie,
                bos_token_id,
            )
        else:
            if append_trie:
                return append_trie.get(prefix_sequence)
            else:
                return [0]
    def __iter__(self):
        def _traverse(prefix_sequence, trie_dict):
            if trie_dict:
                for next_token in trie_dict:
                    yield from _traverse(prefix_sequence + [next_token], trie_dict[next_token])
            else:
                yield prefix_sequence

        return _traverse([], self.trie_dict)
    def __len__(self):
        return self.len
    def __getitem__(self, value):
        return self.get(value)
trie = Trie(entid2text)

In [ ]:

# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True


In [ ]:
import re
from collections import Counter
# from helper import get_performance
import numpy as np
import random

class RunEval:
    def __init__(self, configs, model, tokenizer, ent_name_list, target_embeddings):
        self.configs = configs
        self.ent_name_list = ent_name_list
        self.target_embeddings = target_embeddings
        self.configs = configs
        self.model = model
        self.tokenizer = tokenizer

    @torch.no_grad()
    def validation_step(self, batched_data, dataset_idx):
        input_ids = batched_data['input_ids'].to('cuda')
        attention_mask = batched_data['attention_mask'].to('cuda')
        labels = batched_data['labels']
        labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
        neighboors_embeddings=batched_data['neighboors_embeddings'].to('cuda')
        neighboors_embeddings_mask=batched_data['neighboors_embeddings_mask'].to('cuda')
        target_ent_embeddings=batched_data['target_ent_embeddings'].to('cuda')
        triple_id = batched_data['triplet'].numpy()

        self.get_neigs = get_neigs2 if dataset_idx == 0 else get_neigs0

        old_seqs = []
        ranks = torch.randint(self.configs.num_beams + 1, self.configs.n_ent, (len(labels),))
        for i in range(self.configs.num_beams):
          outputs = self.model.generate(
              input_ids=input_ids,
              attention_mask=attention_mask,
              return_dict_in_generate=True,
              max_length=512,
              # eos_token_id=1,
              prefix_allowed_tokens_fn=lambda batch_idx, m_input_ids: self._next_candidate(batch_idx, m_input_ids, triple_id, dataset_idx, old_seqs),
              # bos_token_id=0,
              neighboors_embeddings=neighboors_embeddings,
              neighboors_embeddings_mask=neighboors_embeddings_mask,
              target_ent_embeddings=target_ent_embeddings,
          )
          pred = outputs.sequences.cpu()
          old_seqs.append(pred)
          pred = pred[:,1:]
          if pred.shape[1] > labels.shape[1]:
            pred = pred[:,:labels.shape[1]]
          else:
            cut_labels = labels[:,:pred.shape[1]]
          cut_labels = labels
          seq_match = (pred==cut_labels).all(1)
          new_ranks = torch.where(~seq_match, ranks, i+1)
          ranks = torch.min(ranks, new_ranks)
        # print(labels)
        # print(outputs.sequences.cpu()[1])
        # print(tokenizer.batch_decode(outputs.sequences.cpu(), skip_special_tokens=True))
        # print(tokenizer.batch_decode(labels, skip_special_tokens=True))
        # raise 'sdfa'

        ranks = ranks.tolist()
        out = {'ranks': ranks}
        return out


    def _next_candidate(self, batch_idx, input_ids, triple_id, dataset_idx, old_seqs=None):
        input_ids = input_ids.cpu()
        # print(input_ids.tolist())
        # next = trie.get(input_ids.tolist())
        # print(next)
        # print('='*30)
        # return next
        if input_ids[-1] == 0 and len(input_ids) != 1:
            return [0]
        pred_ids = self.target_embeddings[triple_id[batch_idx][dataset_idx]]
        pred_id = int(pred_ids[len(input_ids)])
        all_gt_ids = torch.cat(self.get_neigs(triple_id[batch_idx][2-dataset_idx], triple_id[batch_idx][1]))

        all_gt_seq = torch.index_select(self.target_embeddings, 0, all_gt_ids)
        all_gt_seq_mask = (all_gt_seq[:, :len(input_ids)]==input_ids).all(1)
        all_gt_seq_tokens = all_gt_seq[:, len(input_ids)][all_gt_seq_mask]
        if len(old_seqs) > 0:
          old_seq = torch.nn.utils.rnn.pad_sequence([x[batch_idx] for x in old_seqs], batch_first=True, padding_value=0)
          if old_seq.shape[1] > len(input_ids):
            old_seq_mask = (old_seq[:, :len(input_ids)]==input_ids).all(1)
            old_seq_tokens = old_seq[:, len(input_ids)][old_seq_mask]
          else:
            old_seq_tokens = torch.tensor([], dtype=torch.int64)
        else:
          old_seq_tokens = torch.tensor([], dtype=torch.int64)
        all_gt_seq_tokens = set(torch.cat([all_gt_seq_tokens, old_seq_tokens]).tolist())
        pred_id = int(pred_ids[len(input_ids)])
        next_tokens = set(trie.get(input_ids.tolist())).difference(all_gt_seq_tokens)
        if pred_id in all_gt_seq_tokens:
          next_tokens.add(pred_id)
        if len(next_tokens) == 0:
          return [0]
        return list(next_tokens)

    def validation_epoch_end(self, outs):
        pred_tail_out, pred_head_out = outs
        agg_tail_out, agg_head_out, agg_total_out = dict(), dict(), dict()
        for out in pred_tail_out:
            for key, value in out.items():
                if key in agg_tail_out:
                    agg_tail_out[key] += value
                else:
                    agg_tail_out[key] = value
        for out in pred_head_out:
            for key, value in out.items():
                if key in agg_head_out:
                    agg_head_out[key] += value
                else:
                    agg_head_out[key] = value
        tail_ranks, head_ranks = agg_tail_out['ranks'], agg_head_out['ranks']
        del agg_tail_out['ranks']
        del agg_head_out['ranks']
        perf = get_performance(self, head_ranks, tail_ranks)
        print(perf)
        return perf





In [ ]:
import numpy as np
import pandas as pd
def _get_performance(ranks):
    ranks = np.array(ranks, dtype=np.float32)
    out = dict()
    out['mr'] = ranks.mean(axis=0)
    out['mrr'] = (1. / ranks).mean(axis=0)
    out['hit1'] = np.sum(ranks == 1, axis=0) / len(ranks)
    out['hit3'] = np.sum(ranks <= 3, axis=0) / len(ranks)
    out['hit10'] = np.sum(ranks <= 10, axis=0) / len(ranks)
    return out


def get_performance(model, tail_ranks, head_ranks):
    tail_out = _get_performance(tail_ranks)
    head_out = _get_performance(head_ranks)
    mr = np.array([tail_out['mr'], head_out['mr']])
    mrr = np.array([tail_out['mrr'], head_out['mrr']])
    hit1 = np.array([tail_out['hit1'], head_out['hit1']])
    hit3 = np.array([tail_out['hit3'], head_out['hit3']])
    hit10 = np.array([tail_out['hit10'], head_out['hit10']])
    val_mrr = mrr.mean().item()
    perf = {'mrr': mrr, 'mr': mr, 'hit@1': hit1, 'hit@3': hit3, 'hit@10': hit10}
    perf = pd.DataFrame(perf, index=['tail ranking', 'head ranking'])
    perf.loc['mean ranking'] = perf.mean(axis=0)
    for hit in ['hit@1', 'hit@3', 'hit@5', 'hit@10']:
        if hit in list(perf.columns):
            perf[hit] = perf[hit].apply(lambda x: '%.2f%%' % (x * 100))
    return perf



In [ ]:
entid2text_emb = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in entid2text], batch_first=True, padding_value=0)

In [ ]:
entid2text[0]


[0, 6458, 9, 1]

In [ ]:
tokenizer.decode(784)

'['

In [ ]:
import re
from collections import Counter
# from helper import get_performance
import numpy as np
import random

class RunEval:
    def __init__(self, configs, model, tokenizer, ent_name_list, target_embeddings):
        self.configs = configs
        self.ent_name_list = ent_name_list
        self.target_embeddings = target_embeddings
        self.configs = configs
        self.model = model
        self.tokenizer = tokenizer

    @torch.no_grad()
    def validation_step(self, batched_data, dataset_idx):
        input_ids = batched_data['input_ids'].to('cuda')
        attention_mask = batched_data['attention_mask'].to('cuda')
        labels = batched_data['labels']
        labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
        neighboors_embeddings=batched_data['neighboors_embeddings'].to('cuda')
        neighboors_embeddings_mask=batched_data['neighboors_embeddings_mask'].to('cuda')
        target_ent_embeddings=batched_data['target_ent_embeddings'].to('cuda')
        triple_id = batched_data['triplet'].numpy()

        self.get_neigs = get_neigs2 if dataset_idx == 0 else get_neigs0

        old_seqs = []
        ranks = torch.randint(self.configs.num_beams + 1, self.configs.n_ent, (len(labels),))
        for i in range(self.configs.num_beams):
          outputs = self.model.generate(
              input_ids=input_ids,
              attention_mask=attention_mask,
              return_dict_in_generate=True,
              max_length=self.configs.max_length,
              # eos_token_id=1,
              prefix_allowed_tokens_fn=lambda batch_idx, m_input_ids: self._next_candidate(batch_idx, m_input_ids, triple_id, dataset_idx, old_seqs),
              # bos_token_id=0,
              neighboors_embeddings=neighboors_embeddings,
              neighboors_embeddings_mask=neighboors_embeddings_mask,
              target_ent_embeddings=target_ent_embeddings,
          )
          pred = outputs.sequences.cpu()
          old_seqs.append(pred)
          pred = pred[:,1:]
          if pred.shape[1] > labels.shape[1]:
            pred = pred[:,:labels.shape[1]]
            cut_labels = labels
          else:
            cut_labels = labels[:,:pred.shape[1]]
          seq_match = (pred==cut_labels).all(1)
          new_ranks = torch.where(~seq_match, ranks, i+1)
          ranks = torch.min(ranks, new_ranks)
        # print(labels)
        # print(outputs.sequences.cpu()[1])
        # print(tokenizer.batch_decode(outputs.sequences.cpu(), skip_special_tokens=True))
        # print(tokenizer.batch_decode(labels, skip_special_tokens=True))
        # raise 'sdfa'

        ranks = ranks.tolist()
        out = {'ranks': ranks}
        return out


    def _next_candidate(self, batch_idx, input_ids, triple_id, dataset_idx, old_seqs=None):
        input_ids = input_ids.cpu()
        # print(input_ids.tolist())
        # next = trie.get(input_ids.tolist())
        # print(next)
        # print('='*30)
        # return next
        if input_ids[-1] == 0 and len(input_ids) != 1:
            return [0]
        pred_ids = self.target_embeddings[triple_id[batch_idx][dataset_idx]]
        pred_id = int(pred_ids[len(input_ids)])
        all_gt_ids = torch.cat(self.get_neigs(triple_id[batch_idx][2-dataset_idx], triple_id[batch_idx][1]))

        all_gt_seq = torch.index_select(self.target_embeddings, 0, all_gt_ids)
        all_gt_seq_mask = (all_gt_seq[:, :len(input_ids)]==input_ids).all(1)
        all_gt_seq_tokens = all_gt_seq[:, len(input_ids)][all_gt_seq_mask]
        if len(old_seqs) > 0:
          old_seq = torch.nn.utils.rnn.pad_sequence([x[batch_idx] for x in old_seqs], batch_first=True, padding_value=0)
          if old_seq.shape[1] > len(input_ids):
            old_seq_mask = (old_seq[:, :len(input_ids)]==input_ids).all(1)
            old_seq_tokens = old_seq[:, len(input_ids)][old_seq_mask]
          else:
            old_seq_tokens = torch.tensor([], dtype=torch.int64)
        else:
          old_seq_tokens = torch.tensor([], dtype=torch.int64)
        all_gt_seq_tokens = set(torch.cat([all_gt_seq_tokens, old_seq_tokens]).tolist())
        pred_id = int(pred_ids[len(input_ids)])
        next_tokens = set(trie.get(input_ids.tolist())).difference(all_gt_seq_tokens)
        if pred_id in all_gt_seq_tokens:
          next_tokens.add(pred_id)
        if len(next_tokens) == 0:
          return [0]
        return list(next_tokens)

    def validation_epoch_end(self, outs):
        pred_tail_out, pred_head_out = outs
        agg_tail_out, agg_head_out, agg_total_out = dict(), dict(), dict()
        for out in pred_tail_out:
            for key, value in out.items():
                if key in agg_tail_out:
                    agg_tail_out[key] += value
                else:
                    agg_tail_out[key] = value
        for out in pred_head_out:
            for key, value in out.items():
                if key in agg_head_out:
                    agg_head_out[key] += value
                else:
                    agg_head_out[key] = value
        tail_ranks, head_ranks = agg_tail_out['ranks'], agg_head_out['ranks']
        del agg_tail_out['ranks']
        del agg_head_out['ranks']
        perf = get_performance(self, head_ranks, tail_ranks)
        print(perf)
        return perf





## run eval

In [ ]:
from torch.nn.utils.rnn import pad_sequence

class DataCollatorForSeq2Seq:
    model= None
    padding= True
    max_length= None
    pad_to_multiple_of=None
    label_pad_token_id= -100
    data_names = None
    def __init__(self, tokenizer, model=None, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100,data_names=None):
        self.tokenizer = tokenizer
        self.model = model
        self.data_names = data_names
        self.label_pad_token_id = label_pad_token_id

    def __call__(self, features):
        features2 = {}
        for name in self.data_names:
          # if name == 'triplet':
          #   continue
          if name in ['labels','filter_id']:
            padding_value=self.label_pad_token_id
          else:
            padding_value=self.tokenizer.pad_token_id
          x_features = [feature[name] for feature in features]
          features2[name] = torch.nn.utils.rnn.pad_sequence(x_features, batch_first=True, padding_value=padding_value)
        if self.model is not None and hasattr(self.model, "prepare_decoder_input_ids_from_labels"):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features2["labels"])
            features2["decoder_input_ids"] = decoder_input_ids
        return features2


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, data_names=list(train_dataset[0].keys()))


In [ ]:
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

tail_data_loader = DataLoader(tail_test_dataset,
                          batch_size=64,
                          shuffle=False,
                          collate_fn=data_collator,
                          # num_workers=8,
                          pin_memory=True)
head_data_loader = DataLoader(head_test_dataset,
                          batch_size=64,
                          shuffle=False,
                          # num_workers=8,
                          collate_fn=data_collator,
                          pin_memory=True)


class Configs:
    def __init__(self):
        self.num_beams = 1
        # self.num_beam_groups = 1
        self.num_return_sequences = 1
        self.max_length = 30
        self.n_ent = 14541
        self.n_rel = 237

configs = Configs()
runEval = RunEval(configs, model, tokenizer, ent_name_decode_list, entid2text_emb)

runEval.model.eval()
head_list_result = []
for data in tqdm(head_data_loader):
    # pass
    rank_rs = runEval.validation_step(data, 0)
    head_list_result.append(rank_rs)
    # break
tail_list_result = []
for data in tqdm(tail_data_loader):
    rank_rs = runEval.validation_step(data, 2)
    tail_list_result.append(rank_rs)
    # break

kq = runEval.validation_epoch_end((head_list_result, tail_list_result))


  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

                   mrr           mr   hit@1   hit@3  hit@10
tail ranking  0.479574  2367.001953  39.12%  53.62%  67.29%
head ranking  0.353449  3125.134521  26.15%  40.58%  57.04%
mean ranking  0.416512  2746.068359  32.63%  47.10%  62.16%


# EditedT5
<!--  -->

In [ ]:
from transformers import T5ForConditionalGeneration
from transformers import T5Config
import torch
import torch.nn as nn
from typing import Optional, Tuple
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import Seq2SeqLMOutput, BaseModelOutput


def shape(states, batch_size):
    return states.view(batch_size, -1, 8, 64).transpose(1, 2)


class EditedT5(T5ForConditionalGeneration):
  def __init__(self, config: T5Config):
    super().__init__(config)

    self.key_projection = nn.Linear(700, config.d_model)
    self.value_projection1 = nn.Linear(700*2, 700*4)
    self.value_projection2 = nn.Linear(700*4, config.d_model)
    self.act = nn.PReLU()

    self.post_init()

  def k_projection(self, x, i, batch_size):
    return shape(self.decoder.block[i].layer[1].EncDecAttention.k(x), batch_size)

  def v_projection(self, x, i, batch_size):
    return shape(self.decoder.block[i].layer[1].EncDecAttention.v(x), batch_size)

  def forward(
      self,
      input_ids: Optional[torch.LongTensor] = None,
      attention_mask: Optional[torch.FloatTensor] = None,
      decoder_input_ids: Optional[torch.LongTensor] = None,
      decoder_attention_mask: Optional[torch.BoolTensor] = None,
      head_mask: Optional[torch.FloatTensor] = None,
      decoder_head_mask: Optional[torch.FloatTensor] = None,
      cross_attn_head_mask: Optional[torch.Tensor] = None,
      encoder_outputs: Optional[Tuple[Tuple[torch.Tensor]]] = None,
      past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
      inputs_embeds: Optional[torch.FloatTensor] = None,
      decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
      labels: Optional[torch.LongTensor] = None,
      use_cache: Optional[bool] = None,
      output_attentions: Optional[bool] = None,
      output_hidden_states: Optional[bool] = None,
      return_dict: Optional[bool] = None,
      target_ent_embeddings=None,
      neighboors_embeddings=None,
      neighboors_embeddings_mask=None,
  ) :

      value_embeddings = self.value_projection2(self.act(self.value_projection1(neighboors_embeddings)))
      key_embeddings = self.key_projection(target_ent_embeddings) * neighboors_embeddings_mask.unsqueeze(2)
      batch_size = value_embeddings.shape[0]

      use_cache = use_cache if use_cache is not None else self.config.use_cache
      return_dict = return_dict if return_dict is not None else self.config.use_return_dict

      if encoder_outputs is None:
        # Convert encoder inputs in embeddings if needed
        encoder_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
      elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
          encoder_outputs = BaseModelOutput(
              last_hidden_state=encoder_outputs[0],
              hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
              attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
          )
      hidden_states = encoder_outputs[0]


      if past_key_values is None:

        past_key_values = []
        for i in range(self.config.num_layers):
          cross_key = self.k_projection(key_embeddings, i, batch_size)
          cross_value = self.v_projection(value_embeddings, i, batch_size)
          self_key = torch.zeros_like(cross_key)
          self_value = torch.zeros_like(cross_value)
          past_key_values.append((self_key, self_value, cross_key, cross_value))


      if self.model_parallel:
          torch.cuda.set_device(self.decoder.first_device)

      if labels is not None and decoder_input_ids is None and decoder_inputs_embeds is None:
          decoder_input_ids = self._shift_right(labels)

      if self.model_parallel:
          torch.cuda.set_device(self.decoder.first_device)
          hidden_states = hidden_states.to(self.decoder.first_device)
          if decoder_input_ids is not None:
              decoder_input_ids = decoder_input_ids.to(self.decoder.first_device)
          if attention_mask is not None:
              attention_mask = attention_mask.to(self.decoder.first_device)
          if decoder_attention_mask is not None:
              decoder_attention_mask = decoder_attention_mask.to(self.decoder.first_device)

      # Decode
      decoder_outputs = self.decoder(
          input_ids=decoder_input_ids,
          attention_mask=decoder_attention_mask,
          inputs_embeds=decoder_inputs_embeds,
          past_key_values=past_key_values,
          encoder_hidden_states=hidden_states,
          encoder_attention_mask=attention_mask,
          head_mask=decoder_head_mask,
          cross_attn_head_mask=cross_attn_head_mask,
          use_cache=use_cache,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )

      sequence_output = decoder_outputs[0]

      # Set device for model parallelism
      if self.model_parallel:
          torch.cuda.set_device(self.encoder.first_device)
          self.lm_head = self.lm_head.to(self.encoder.first_device)
          sequence_output = sequence_output.to(self.lm_head.weight.device)

      if self.config.tie_word_embeddings:
          sequence_output = sequence_output * (self.model_dim**-0.5)

      lm_logits = self.lm_head(sequence_output)

      loss = None
      if labels is not None:
          loss_fct = CrossEntropyLoss(ignore_index=-100)
          # move labels to correct device to enable PP
          labels = labels.to(lm_logits.device)
          loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))

      if not return_dict:
          output = (lm_logits,) + decoder_outputs[1:] + encoder_outputs
          return ((loss,) + output) if loss is not None else output

      return Seq2SeqLMOutput(
          loss=loss,
          logits=lm_logits,
          past_key_values=decoder_outputs.past_key_values,
          decoder_hidden_states=decoder_outputs.hidden_states,
          decoder_attentions=decoder_outputs.attentions,
          cross_attentions=decoder_outputs.cross_attentions,
          encoder_last_hidden_state=encoder_outputs.last_hidden_state,
          encoder_hidden_states=encoder_outputs.hidden_states,
          encoder_attentions=encoder_outputs.attentions,
      )

  def prepare_inputs_for_generation(
        self,
        input_ids,
        past_key_values=None,
        attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        decoder_attention_mask=None,
        cross_attn_head_mask=None,
        use_cache=None,
        encoder_outputs=None,
        target_ent_embeddings=None,
        neighboors_embeddings=None,
        neighboors_embeddings_mask=None,
        **kwargs,
    ):
        # cut decoder_input_ids if past_key_values is used
        if past_key_values is not None:
            past_length = past_key_values[0][0].shape[2]

            # Some generation methods already pass only the last input ID
            if input_ids.shape[1] > past_length:
                remove_prefix_length = past_length
            else:
                # Default to old behavior: keep only final ID
                remove_prefix_length = input_ids.shape[1] - 1

            input_ids = input_ids[:, remove_prefix_length:]

        return {
            "decoder_input_ids": input_ids,
            "past_key_values": past_key_values,
            "encoder_outputs": encoder_outputs,
            "attention_mask": attention_mask,
            "head_mask": head_mask,
            "decoder_head_mask": decoder_head_mask,
            "decoder_attention_mask": decoder_attention_mask,
            "cross_attn_head_mask": cross_attn_head_mask,
            "use_cache": use_cache,
            "target_ent_embeddings": target_ent_embeddings,
            "neighboors_embeddings": neighboors_embeddings,
            "neighboors_embeddings_mask": neighboors_embeddings_mask,
            # "input_ids": input_ids,
        }

# write dataset


In [ ]:
from ampligraph.datasets import load_fb15k_237
from ampligraph.pretrained_models import load_pretrained_model

model = load_pretrained_model(dataset="fb15k-237", scoring_type="RotatE")


Saved model does not include a db file. Skipping.


In [ ]:
# ent2id
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/entities.txt'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entities.txt
path = "/content/entities.txt"

ent2id = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    # print(line.strip().split('\t'))
    ent = line.strip().split('\t')[0]
    ent2id[ent] = int(i)


Processing lines:   0%|          | 0/14541 [00:00<?, ?it/s]

In [ ]:
# rel2id
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/relations.txt'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/relations.txt
path = "/content/relations.txt"

rel2id = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    # print(line.strip().split('\t'))
    rel = line.strip().split('\t')[0]
    rel2id[rel] = int(i)


Processing lines:   0%|          | 0/237 [00:00<?, ?it/s]

In [ ]:
%%capture
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small', padding=True)

def _tokenize( x):
    return tokenizer(x, return_tensors="pt")['input_ids'][0][:-1]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# ent2text
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/entity2text.txt'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entity2text.txt
path = "/content/entity2text.txt"

ent2text = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    ent, text = line.strip().split('\t')
    ent2text[ent] = _tokenize(text)


--2024-04-14 17:41:33--  https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entity2text.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zjunlp/Relphormer/main/dataset/fb15k-237/entity2text.txt [following]
--2024-04-14 17:41:33--  https://raw.githubusercontent.com/zjunlp/Relphormer/main/dataset/fb15k-237/entity2text.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 392383 (383K) [text/plain]
Saving to: ‘entity2text.txt’

entity2text.txt     100%[===================>] 383.19K  --.-KB/s    in 0.006s  

2024-04-14 17:41:34 (60.1 MB/s) - ‘entity2text.txt’ saved [392383/392383]


Processing lines:   0%|          | 0/14951 [00:00<?, ?it/s]

In [ ]:
# rel2text
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/relation2text.txt'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/relation2text.txt
path = "/content/relation2text.txt"

rel2text = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    rel, text = line.strip().split('\t')
    rel2text[rel] = _tokenize(text)


Processing lines:   0%|          | 0/237 [00:00<?, ?it/s]

In [ ]:
# ent2decs
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/entity2textlong.txt'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/entity2textlong.txt
path = "/content/entity2textlong.txt"

ent2decs = {}
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    ent, text = line.strip().split('\t')
    ent2decs[ent] = _tokenize(text)[:64]


Processing lines:   0%|          | 0/14951 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# train triplet
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/train.tsv'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/train.tsv
path = "/content/train.tsv"

train_triplet_id = []
train_triplet_tokens = []
train_triplet_decs = []
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    head, relation, tail = line.strip().split('\t')
    head_id = ent2id[head]
    relation_id = rel2id[relation]
    tail_id = ent2id[tail]
    head_tokens = ent2text[head]
    relation_tokens = rel2text[relation]
    tail_tokens = ent2text[tail]
    head_decs = ent2decs[head]
    tail_decs = ent2decs[tail]
    train_triplet_id.append((head_id, relation_id, tail_id))
    train_triplet_tokens.append((head_tokens, relation_tokens, tail_tokens))
    train_triplet_decs.append((head_decs, tail_decs))


Processing lines:   0%|          | 0/272115 [00:00<?, ?it/s]

In [ ]:
# valid triplet
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/dev.tsv'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/dev.tsv
path = "/content/dev.tsv"

valid_triplet_id = []
valid_triplet_tokens = []
valid_triplet_decs = []
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    head, relation, tail = line.strip().split('\t')
    head_id = ent2id[head]
    relation_id = rel2id[relation]
    tail_id = ent2id[tail]
    head_tokens = ent2text[head]
    relation_tokens = rel2text[relation]
    tail_tokens = ent2text[tail]
    head_decs = ent2decs[head]
    tail_decs = ent2decs[tail]
    valid_triplet_id.append((head_id, relation_id, tail_id))
    valid_triplet_tokens.append((head_tokens, relation_tokens, tail_tokens))
    valid_triplet_decs.append((head_decs, tail_decs))

Processing lines:   0%|          | 0/17535 [00:00<?, ?it/s]

In [ ]:
# test triplet
from tqdm.auto import tqdm
import os

if not os.path.exists('/content/test.tsv'):
  !wget https://github.com/zjunlp/Relphormer/raw/main/dataset/fb15k-237/test.tsv
path = "/content/test.tsv"

test_triplet_id = []
test_triplet_tokens = []
test_triplet_decs = []
with open(path, "r") as f:
  total_lines = sum(1 for _ in f)
  f.seek(0)  # Reset file pointer to the beginning
  for i, line in tqdm(enumerate(f), total=total_lines, desc="Processing lines"):
    head, relation, tail = line.strip().split('\t')
    head_id = ent2id[head]
    relation_id = rel2id[relation]
    tail_id = ent2id[tail]
    head_tokens = ent2text[head]
    relation_tokens = rel2text[relation]
    tail_tokens = ent2text[tail]
    head_decs = ent2decs[head]
    tail_decs = ent2decs[tail]
    test_triplet_id.append((head_id, relation_id, tail_id))
    test_triplet_tokens.append((head_tokens, relation_tokens, tail_tokens))
    test_triplet_decs.append((head_decs, tail_decs))

Processing lines:   0%|          | 0/20466 [00:00<?, ?it/s]

NameError: name 'model' is not defined

In [ ]:
id2ent_rotatE = model.data_indexer.backend.entities_dict
RotatE_ent = torch.load("/content/RotatE_ent.pt")

new_RotatE_ent_dict = {}
for rid, rent in id2ent_rotatE.items():
  emb = RotatE_ent[rid]
  ent_id = ent2id[rent]
  new_RotatE_ent_dict[ent_id] = emb

new_RotatE_ent = []
for i in range(len(id2ent)):
  if i in new_RotatE_ent_dict:
    new_RotatE_ent.append(new_RotatE_ent_dict[i])

new_RotatE_ent = torch.stack(new_RotatE_ent)

id2rel_rotateE = model.data_indexer.backend.relations_dict
RotatE_rel = torch.load("/content/RotatE_rel.pt")

new_RotatE_rel_dict = {}
for rid, rrel in id2rel_rotateE.items():
  emb = RotatE_rel[rid]
  rel_id = rel2id[rrel]
  new_RotatE_rel_dict[rel_id] = emb

new_RotatE_rel = []
for i in range(len(id2rel)):
  if i in new_RotatE_rel_dict:
    new_RotatE_rel.append(new_RotatE_rel_dict[i])

new_RotatE_rel = torch.stack(new_RotatE_rel)

In [ ]:
kgt5_data = {
    "train_triplet_id":train_triplet_id,
    "train_triplet_tokens":train_triplet_tokens,
    "train_triplet_decs":train_triplet_decs,
    "valid_triplet_id":valid_triplet_id,
    "valid_triplet_tokens":valid_triplet_tokens,
    "valid_triplet_decs":valid_triplet_decs,
    "test_triplet_id":test_triplet_id,
    "test_triplet_tokens":test_triplet_tokens,
    "test_triplet_decs":test_triplet_decs,
    "RotatE_ent_emb":new_RotatE_ent,
    "RotatE_rel_emb":new_RotatE_rel,
}

In [ ]:
import torch
torch.save(kgt5_data, "/content/kgt5_data.pt")
!gsutil cp kgt5_data.pt gs://hien7613storage2/

Copying file://kgt5_data.pt [Content-Type=application/vnd.snesdev-page-table]...
|
Operation completed over 1 objects/87.0 MiB.                                     


In [ ]:
kgt5_data.keys()

dict_keys(['train_triplet_id', 'train_triplet_tokens', 'train_triplet_decs', 'valid_triplet_id', 'valid_triplet_tokens', 'valid_triplet_decs', 'test_triplet_id', 'test_triplet_tokens', 'test_triplet_decs', 'RotatE_ent_emb', 'RotatE_rel_emb'])